In [1]:
import math
import time
import torch 
import numpy as np
import torch
import numpy as np
from ogb.graphproppred import PygGraphPropPredDataset
from ogb.lsc.pcqm4mv2_pyg import PygPCQM4Mv2Dataset
from functools import lru_cache
import pyximport
import torch.distributed as dist
import time
pyximport.install(setup_args={"include_dirs": np.get_include()})
from mydata_abakan import geo_Abakan
from mydata_omsk import geo_Omsk


import algos



@torch.jit.script
def convert_to_single_emb(x, offset: int = 512):
    feature_num = x.size(1) if len(x.size()) > 1 else 1
    feature_offset = 1 + torch.arange(0, feature_num * offset, offset, dtype=torch.long)
    x = x + feature_offset
    return x


def preprocess_item(item, short_path_mode = 'floyd'):
    
    # print('preprocess start')
    start = time.time()
    edge_attr, edge_index, x = item.edge_attr, item.edge_index, item.x
    # print('edge_attr size', edge_attr.size()) 
    # print('edge_index size', edge_index.size()) 
    # print('x size', x.size()) 
    N = x.size(0)
    x = convert_to_single_emb(x)

    # node adj matrix [N, N] bool
    adj = torch.zeros([N, N], dtype=torch.bool)
    adj[edge_index[0, :], edge_index[1, :]] = True

    # edge feature here
    if len(edge_attr.size()) == 1:
        edge_attr = edge_attr[:, None]
    attn_edge_type = torch.zeros([N, N, edge_attr.size(-1)], dtype=torch.long)
    attn_edge_type[edge_index[0, :], edge_index[1, :]] = (
        convert_to_single_emb(edge_attr) + 1
    )
    
    start_floyd = time.time()
    if short_path_mode == 'floyd':
        print('start find path floyd')
        start_algo= time.time()
        shortest_path_result, path = algos.floyd_warshall(adj.numpy())
        # print('shortest_path_result:', shortest_path_result)
        # print('path:', path)
        # print('type shortest_path_result:', type(shortest_path_result))
        # print('type path:', type(path))
        # print('size shortest_path_result:', shortest_path_result.shape)
        # print('size path:', path.shape)
        # time.sleep(100)

        end_algo = time.time()
        print('floyd end with time', end_algo - start_algo)
        
        max_dist = np.amax(shortest_path_result)
        
        print('start gen_edge_input floyd')
        start_gen_edge_input= time.time()
        edge_input = algos.gen_edge_input(max_dist, path, attn_edge_type.numpy())
        end_gen_edge_input = time.time()
        print('gen_edge_input floyd end with time', end_gen_edge_input - start_gen_edge_input)
    elif short_path_mode == 'dijkstra':
        print('start find path dijkstra')
        start_algo= time.time()
        shortest_path_result, path = algos.algo_dijk(adj.numpy())
        end_algo = time.time()
        print('dijkstra end with time', end_algo - start_algo)
        max_dist = np.amax(shortest_path_result)
        
        print('start gen_edge_input dijkstra')
        start_gen_edge_input= time.time()
        edge_input = algos.gen_edge_input_dijk(max_dist, path, attn_edge_type.numpy())
        end_gen_edge_input = time.time()
        print('gen_edge_input dijkstra end with time', end_gen_edge_input - start_gen_edge_input)
    elif short_path_mode == 'bfs':
        print('start find path bfs')
        start_algo= time.time()
        shortest_path_result, path = algos.floyd_warshall(adj.numpy())
        end_algo = time.time()
        print('bfs end with time', end_algo - start_algo)
        
        max_dist = np.amax(shortest_path_result)
        
        print('start gen_edge_input bfs')
        start_gen_edge_input= time.time()
        edge_input = algos.gen_edge_input_dijk(max_dist, path, attn_edge_type.numpy())
        end_gen_edge_input = time.time()
        print('gen_edge_input bfs end with time', end_gen_edge_input - start_gen_edge_input)
    spatial_pos = torch.from_numpy((shortest_path_result)).long()
    attn_bias = torch.zeros([N + 1, N + 1], dtype=torch.float)  # with graph token

    # combine
    item.x = x
    item.attn_bias = attn_bias
    item.attn_edge_type = attn_edge_type
    item.spatial_pos = spatial_pos
    item.in_degree = adj.long().sum(dim=1).view(-1)
    item.out_degree = item.in_degree  # for undirected graph
    item.edge_input = torch.from_numpy(edge_input).long()
    
    end = time.time()
    return item











def arg_min(T, S):
    amin = -1
    m = math.inf  # максимальное значение
    for i, t in enumerate(T):
        if t < m and i not in S:
            m = t
            amin = i

    return amin

def prepare_adj(D):
    N = len(D) 
    mask = (torch.ones(N,N, dtype=torch.float) - torch.eye(N,N, dtype=torch.float))*500
    return torch.where(D > 0, D, mask)

def algo_dijk(D):
    N = len(D)  # число вершин в графе
    T = [math.inf]*N   # последняя строка таблицы
    path = list()
    distance_matrix = np.zeros([N, N])
    path_matrix = np.zeros([N, N])
    for i in range(N):
        for j in range(N):
            if i == j:
                D[i][j] = 0
            elif D[i][j] == 0:
                D[i][j] = 510
    for node in range(N):
        v = node     # стартовая вершина (нумерация с нуля)
        S = {v}     # просмотренные вершины
        T = [math.inf]*N   # последняя строка таблицы
        T[v] = 0    # нулевой вес для стартовой вершины
        M = [0]*N   # оптимальные связи между вершинами

        while v != -1:          # цикл, пока не просмотрим все вершины
            for j, dw in enumerate(D[v]):   # перебираем все связанные вершины с вершиной v
                if j not in S:           # если вершина еще не просмотрена
                    w = T[v] + dw
                    if w < T[j]:
                        T[j] = w
                        M[j] = v        # связываем вершину j с вершиной v

            v = arg_min(T, S)            # выбираем следующий узел с наименьшим весом
            if v >= 0:                    # выбрана очередная вершина
                S.add(v)                 # добавляем новую вершину в рассмотрение

    #print(T, M, sep="\n")
        
        distance_matrix[node] = T
        path_matrix[node] = M

        
    return(distance_matrix, path_matrix)

def get_all_edges_dijk(path, i, j):
    # формирование оптимального маршрута:
    start = i
    end = j
    P = [j]
    while end != start:
        end = int(path[i][P[-1]])
        P.append(end)
    return P


def gen_edge_input(max_dist, path, edge_feat):
    # print('start gen_edge_input')
    (nrows, ncols) = path.shape
    assert nrows == ncols
    n = nrows
    # print('max_dist', max_dist)
    max_dist_copy = int(max_dist)
    # print('max_dist_copy', max_dist_copy)
    
    path_copy = path
    edge_feat_copy = edge_feat
    
    # print('start numpy')
    start_numpy = time.time()
    edge_fea_all = -1 * np.ones([n, n, max_dist_copy, edge_feat.shape[-1]])
    
#     print('edge_fea_all shape', edge_feat.shape[-1])
#     print('edge_fea_all', edge_fea_all)
    
    end_numpy= time.time()
    # print('end numpy')
    # print('numpy end with time', end_numpy - start_numpy)
        
    # print('start sycle')
    start_sycle = time.time()
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            if path_copy[i][j] == 510:
                continue
            path = algos.get_all_edges_dijk(path_copy, i, j)
            # path = [i] + [j]
            num_path = len(path) - 1
            # print(path)
            for k in range(num_path):
                # print(i)
                # print(j)
                # print(k)
                # print(edge_fea_all)
                edge_fea_all[i, j, k, :] = edge_feat_copy[path[k], path[k+1], :]
#                 edge_fea_all[i, j, k, :] = edge_feat_copy[path[0], path[1], :]
# #                 
    end_sycle = time.time()
    # print('end sycle')
    # print('sycle end with time', end_sycle - start_sycle)
    
    # print('end gen_edge_input')
    return edge_fea_all

Using backend: pytorch


In [2]:
root = '/home/jovyan/graphormer_v2/examples/georides/omsk/dataset/omsk'
data = geo_Omsk(root = root)

In [3]:
preprocess_item(data[2], short_path_mode = 'dijkstra')

start find path dijkstra
dijkstra end with time 0.06913995742797852
start gen_edge_input dijkstra
gen_edge_input dijkstra end with time 0.05920124053955078


Data(attn_bias=[89, 89], attn_edge_type=[88, 88, 1], edge_attr=[88, 1], edge_index=[2, 88], edge_input=[88, 88, 87, 1], in_degree=[88], out_degree=[88], spatial_pos=[88, 88], x=[88, 13], y=[1])

In [4]:
preprocess_item(data[2], short_path_mode = 'floyd')

start find path floyd
floyd end with time 0.004880189895629883
start gen_edge_input floyd
gen_edge_input floyd end with time 0.3409566879272461


Data(attn_bias=[89, 89], attn_edge_type=[88, 88, 1], edge_attr=[88, 1], edge_index=[2, 88], edge_input=[88, 88, 87, 1], in_degree=[88], out_degree=[88], spatial_pos=[88, 88], x=[88, 13], y=[1])